In [1]:
# Model Building and Evaluation for Solar Panel Detection
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO
import supervision as sv
from supervision.metrics.detection import MeanAveragePrecision
from supervision.metrics.detection import ConfusionMatrix
from tqdm.auto import tqdm
import pandas as pd
import yaml
import seaborn as sns
from sklearn.model_selection import train_test_split
from pathlib import Path
import rasterio
import shutil

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Create base directory for the project
base_dir = os.path.abspath("solar_panel_project")
os.makedirs(base_dir, exist_ok=True)

In [2]:
# Data Preparation and Splitting
labels_dir = "labels_native"
labels_hd_dir = "labels_hd"  # Not used in this script but kept for reference
images_dir = "image_chips_native"

# Get all image and label files
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.tif')])
label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.txt')])

print(f"Total images: {len(image_files)}")
print(f"Total label files: {len(label_files)}")

# Create matched pairs of images and labels
matched_files = []
for img_file in image_files:
    base_name = os.path.splitext(img_file)[0]
    label_file = base_name + '.txt'
    if label_file in label_files:
        matched_files.append((img_file, label_file))

print(f"Matched image-label pairs: {len(matched_files)}")

# Split data into train, validation, and test sets
train_val_files, test_files = train_test_split(matched_files, test_size=0.2, random_state=42)
train_files, val_files = train_test_split(train_val_files, test_size=0.125, random_state=42)  # 10% of original data

print(f"Train files: {len(train_files)}")
print(f"Validation files: {len(val_files)}")
print(f"Test files: {len(test_files)}")

# Create directories for the dataset splits
os.makedirs(os.path.join(base_dir, 'dataset/images/train'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'dataset/images/val'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'dataset/images/test'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'dataset/labels/train'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'dataset/labels/val'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'dataset/labels/test'), exist_ok=True)

# Copy files to their respective directories (using shutil instead of os.system)
def copy_files(file_pairs, split_type):
    for img_file, label_file in file_pairs:
        # Copy image
        img_src = os.path.join(images_dir, img_file)
        img_dst = os.path.join(base_dir, f'dataset/images/{split_type}/{img_file}')
        shutil.copy2(img_src, img_dst)
        
        # Copy label
        lbl_src = os.path.join(labels_dir, label_file)
        lbl_dst = os.path.join(base_dir, f'dataset/labels/{split_type}/{label_file}')
        shutil.copy2(lbl_src, lbl_dst)

copy_files(train_files, 'train')
copy_files(val_files, 'val')
copy_files(test_files, 'test')

# Create dataset.yaml file for YOLO training
dataset_config = {
    'path': os.path.join(base_dir, 'dataset'),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'names': {0: 'solar_panel'}  # Single class - solar panels
}

with open(os.path.join(base_dir, 'dataset.yaml'), 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False)

Total images: 2553
Total label files: 2542
Matched image-label pairs: 2542
Train files: 1778
Validation files: 255
Test files: 509


In [4]:
# Train YOLO Model
# Initialize YOLO model - you can choose different model sizes based on your needs
model = YOLO('yolov8n.pt')  # Using nano model, can also use 'yolov8s.pt', 'yolov8m.pt', etc.

# Train the model
results = model.train(
    data=os.path.join(base_dir, 'dataset.yaml'),
    epochs=25,
    imgsz=416,
    patience=10,  # Early stopping
    batch=16,
    device='cpu',
    save=True,
    name='solar_panel_detection'
)

Ultralytics 8.3.86  Python-3.11.7 torch-2.4.1+cpu CPU (12th Gen Intel Core(TM) i5-1240P)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset.yaml, epochs=25, time=None, patience=10, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=solar_panel_detection3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=

train: Scanning C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\labels\train.cache

train: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\train\solarpanels_native_1__x0_0_y0_6963_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\train\solarpanels_native_1__x0_0_y0_7229_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\train\solarpanels_native_1__x0_0_y0_7249_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\train\solarpanels_native_1__x0_1300_y0_10050_dxdy_416.tif: ignoring corrupt image/label:


val: Scanning C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\labels\val.cache... 

val: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\val\solarpanels_native_1__x0_0_y0_7119_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\val\solarpanels_native_1__x0_0_y0_7230_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\val\solarpanels_native_1__x0_128_y0_7056_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\val\solarpanels_native_1__x0_1501_y0_10325_dxdy_416.tif: ignoring corrupt image/label: Label class 2

...

      25/25         0G     0.8802      0.522     0.9092        138        416: 100%|██████████| 101/101 [08:17<00:00,  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:24<0

                   all        226       2738       0.94      0.893      0.955      0.735



25 epochs completed in 9.399 hours.
Optimizer stripped from runs\detect\solar_panel_detection3\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\solar_panel_detection3\weights\best.pt, 6.2MB

Validating runs\detect\solar_panel_detection3\weights\best.pt...
Ultralytics 8.3.86  Python-3.11.7 torch-2.4.1+cpu CPU (12th Gen Intel Core(TM) i5-1240P)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:22<0


                   all        226       2738      0.939      0.893      0.954      0.735
Speed: 2.3ms preprocess, 67.7ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\solar_panel_detection3


In [24]:
# Import required libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import random
import cv2
import supervision as sv
from supervision.metrics.detection import MeanAveragePrecision
from supervision.metrics.detection import ConfusionMatrix
from supervision.detection.utils import clip_boxes
import torch
from tqdm import tqdm
import pandas as pd

# Load the trained model
model_path = "runs/detect/solar_panel_detection/weights/best.pt"
model = YOLO(model_path)

# 1. Show validation loss convergence
def plot_validation_loss():
    # Load the results file that was saved during training
    results = model.info()
    
    # Check if 'results' is a list or a dictionary
    if isinstance(results, list):
        val_losses = [result.get('val/box_loss', 0) + result.get('val/cls_loss', 0) + result.get('val/dfl_loss', 0) 
                      for result in results]
        epochs = list(range(1, len(val_losses) + 1))
    else:
        # If it's a dictionary, try to extract metrics directly
        val_box_loss = results.get('val/box_loss', [])
        val_cls_loss = results.get('val/cls_loss', [])
        val_loss = results.get('val/loss', [])
        epochs = list(range(1, len(val_loss) + 1))
    
    # Plot the validation loss
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.plot(epochs, train_box_loss, label='Training Box Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Validation Loss Convergence')
    plt.legend()
    plt.grid(True)
    plt.savefig('validation_loss_convergence.png')
    plt.show()
    
    print(f"Final validation loss: {val_loss[-1]}")
    return val_loss

# 2. Visualize ground truth vs. predictions
def visualize_predictions(test_loader, num_samples=4):
    # Get a list of test images
    test_images = list(test_loader.dataset.img_files)
    
    # Select random images from the test set
    random_indices = random.sample(range(len(test_images)), num_samples)
    selected_images = [test_images[i] for i in random_indices]
    
    fig, axes = plt.subplots(num_samples, 2, figsize=(15, 5*num_samples))
    
    for i, img_path in enumerate(selected_images):
        # Load the image - handle .TIF files correctly
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        
        # Handle TIF images which might have more than 3 channels or different bit depths
        if img is None:
            # For TIF images that cv2 can't handle, try using PIL
            from PIL import Image
            pil_img = Image.open(img_path)
            img = np.array(pil_img)
            
            # If image has more than 3 channels, use only the first 3
            if len(img.shape) > 2 and img.shape[2] > 3:
                img = img[:, :, :3]
            
            # If it's grayscale, convert to RGB
            if len(img.shape) == 2:
                img = np.stack([img, img, img], axis=2)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Get ground truth annotations
        # Parse the corresponding label file
        label_path = img_path.replace('images', 'labels').replace('.tif', '.txt').replace('.TIF', '.txt')
        gt_boxes = []
        gt_classes = []
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    data = line.strip().split()
                    class_id = int(data[0])
                    x_center, y_center, width, height = map(float, data[1:5])
                    
                    # Convert YOLO format to pixel coordinates
                    img_height, img_width = img.shape[:2]
                    x1 = int((x_center - width/2) * img_width)
                    y1 = int((y_center - height/2) * img_height)
                    x2 = int((x_center + width/2) * img_width)
                    y2 = int((y_center + height/2) * img_height)
                    
                    gt_boxes.append([x1, y1, x2, y2])
                    gt_classes.append(class_id)
        
        # Get model predictions
        results = model(img)
        pred_boxes = []
        pred_scores = []
        pred_classes = []
        
        if results[0].boxes:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            scores = results[0].boxes.conf.cpu().numpy()
            cls = results[0].boxes.cls.cpu().numpy().astype(int)
            
            for box, score, cl in zip(boxes, scores, cls):
                pred_boxes.append(box)
                pred_scores.append(score)
                pred_classes.append(cl)
        
        # Plot ground truth
        img_gt = img.copy()
        for box, cl in zip(gt_boxes, gt_classes):
            x1, y1, x2, y2 = box
            cv2.rectangle(img_gt, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green for ground truth
            cv2.putText(img_gt, f"Class {cl}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Plot predictions
        img_pred = img.copy()
        for box, score, cl in zip(pred_boxes, pred_scores, pred_classes):
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(img_pred, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue for predictions
            cv2.putText(img_pred, f"Class {cl}: {score:.2f}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
        # Display images
        axes[i, 0].imshow(img_gt)
        axes[i, 0].set_title(f"Ground Truth - Image {i+1}")
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(img_pred)
        axes[i, 1].set_title(f"Predictions - Image {i+1}")
        axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig('ground_truth_vs_predictions.png')
    plt.show()

# Function to load TIF images correctly
def load_image(img_path):
    # Try standard OpenCV reading first
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    
    # If that fails or returns None, try with PIL
    if img is None:
        from PIL import Image
        pil_img = Image.open(img_path)
        img = np.array(pil_img)
        
        # If image has more than 3 channels, use only the first 3
        if len(img.shape) > 2 and img.shape[2] > 3:
            img = img[:, :, :3]
        
        # If it's grayscale, convert to RGB
        if len(img.shape) == 2:
            img = np.stack([img, img, img], axis=2)
    else:
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    return img

# 3. Compute mAP50 using supervision
def compute_map50(test_loader):
    # Define the metrics calculator
    metrics_calculator = StaticDetectionMetricsCalculator()
    
    print("Computing mAP50...")
    for img_path in tqdm(test_loader.dataset.img_files):
        # Load the image
        img = load_image(img_path)
        
        # Get ground truth annotations
        label_path = img_path.replace('images', 'labels').replace('.tif', '.txt').replace('.TIF', '.txt')
        gt_boxes = []
        gt_classes = []
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    data = line.strip().split()
                    class_id = int(data[0])
                    x_center, y_center, width, height = map(float, data[1:5])
                    
                    # Convert YOLO format to pixel coordinates
                    img_height, img_width = img.shape[:2]
                    x1 = int((x_center - width/2) * img_width)
                    y1 = int((y_center - height/2) * img_height)
                    x2 = int((x_center + width/2) * img_width)
                    y2 = int((y_center + height/2) * img_height)
                    
                    gt_boxes.append([x1, y1, x2, y2])
                    gt_classes.append(class_id)
        
        # Get model predictions
        results = model(img)
        pred_boxes = []
        pred_scores = []
        pred_classes = []
        
        if results[0].boxes:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            scores = results[0].boxes.conf.cpu().numpy()
            cls = results[0].boxes.cls.cpu().numpy().astype(int)
            
            for box, score, cl in zip(boxes, scores, cls):
                pred_boxes.append(box)
                pred_scores.append(score)
                pred_classes.append(cl)
        
        # Convert to supervision detections format
        gt_detections = sv.Detections(
            xyxy=np.array(gt_boxes) if gt_boxes else np.zeros((0, 4)),
            class_id=np.array(gt_classes) if gt_classes else np.zeros(0),
            confidence=np.ones(len(gt_classes)) if gt_classes else np.zeros(0)
        )
        
        pred_detections = sv.Detections(
            xyxy=np.array(pred_boxes) if pred_boxes else np.zeros((0, 4)),
            class_id=np.array(pred_classes) if pred_classes else np.zeros(0),
            confidence=np.array(pred_scores) if pred_scores else np.zeros(0)
        )
        
        # Update metrics
        metrics_calculator.update(
            groundtruth=gt_detections,
            prediction=pred_detections,
            image_id=img_path
        )
    
    # Calculate metrics
    metrics = metrics_calculator.get_metrics(iou_threshold=0.5)
    
    print(f"mAP50 (using supervision): {metrics.map}")
    
    # You can also get per-class metrics
    for class_id, class_metrics in metrics.class_metrics.items():
        print(f"Class {class_id} - AP: {class_metrics.ap}, Precision: {class_metrics.precision}, Recall: {class_metrics.recall}")
    
    return metrics

# 4. Create P/R/F1 table with different IoU and confidence thresholds
def create_metrics_table(test_loader):
    # Define the IoU and confidence thresholds
    iou_thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]
    confidence_thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]
    
    # Initialize result dictionaries
    precision_results = {}
    recall_results = {}
    f1_results = {}
    
    # Create confusion matrix calculator
    confusion_matrix = ConfusionMatrix(
        num_classes=1,  # Assuming single class detection for solar panels
        iou_threshold=0.5  # This will be updated inside the loop
    )
    
    # Collect all ground truth and predictions
    all_gt_detections = []
    all_pred_detections = []
    all_image_ids = []
    
    print("Collecting detections for metrics calculation...")
    for img_path in tqdm(test_loader.dataset.img_files):
        # Load the image
        img = load_image(img_path)
        img_height, img_width = img.shape[:2]
        
        # Get ground truth annotations
        label_path = img_path.replace('images', 'labels').replace('.tif', '.txt').replace('.TIF', '.txt')
        gt_boxes = []
        gt_classes = []
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    data = line.strip().split()
                    class_id = int(data[0])
                    x_center, y_center, width, height = map(float, data[1:5])
                    
                    # Convert YOLO format to pixel coordinates
                    x1 = int((x_center - width/2) * img_width)
                    y1 = int((y_center - height/2) * img_height)
                    x2 = int((x_center + width/2) * img_width)
                    y2 = int((y_center + height/2) * img_height)
                    
                    gt_boxes.append([x1, y1, x2, y2])
                    gt_classes.append(class_id)
        
        # Get model predictions
        results = model(img)
        pred_boxes = []
        pred_scores = []
        pred_classes = []
        
        if results[0].boxes:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            scores = results[0].boxes.conf.cpu().numpy()
            cls = results[0].boxes.cls.cpu().numpy().astype(int)
            
            for box, score, cl in zip(boxes, scores, cls):
                pred_boxes.append(box)
                pred_scores.append(score)
                pred_classes.append(cl)
        
        # Convert to supervision detections format
        gt_detection = sv.Detections(
            xyxy=np.array(gt_boxes) if gt_boxes else np.zeros((0, 4)),
            class_id=np.array(gt_classes) if gt_classes else np.zeros(0),
            confidence=np.ones(len(gt_classes)) if gt_classes else np.zeros(0)
        )
        
        pred_detection = sv.Detections(
            xyxy=np.array(pred_boxes) if pred_boxes else np.zeros((0, 4)),
            class_id=np.array(pred_classes) if pred_classes else np.zeros(0),
            confidence=np.array(pred_scores) if pred_scores else np.zeros(0)
        )
        
        all_gt_detections.append(gt_detection)
        all_pred_detections.append(pred_detection)
        all_image_ids.append(img_path)
    
    # For each combination of IoU and confidence threshold
    for iou_threshold in iou_thresholds:
        precision_row = {}
        recall_row = {}
        f1_row = {}
        
        for conf_threshold in confidence_thresholds:
            total_tp = 0
            total_fp = 0
            total_fn = 0
            
            # Process each image with these thresholds
            for gt_detection, pred_detection, img_id in zip(all_gt_detections, all_pred_detections, all_image_ids):
                # Filter predictions by confidence threshold
                confidence_filter = pred_detection.confidence >= conf_threshold
                filtered_pred = sv.Detections(
                    xyxy=pred_detection.xyxy[confidence_filter] if any(confidence_filter) else np.zeros((0, 4)),
                    class_id=pred_detection.class_id[confidence_filter] if any(confidence_filter) else np.zeros(0),
                    confidence=pred_detection.confidence[confidence_filter] if any(confidence_filter) else np.zeros(0)
                )
                
                # Update confusion matrix with the current IoU threshold
                confusion_matrix.iou_threshold = iou_threshold
                confusion_matrix.process_image(
                    gt_detections=gt_detection,
                    pred_detections=filtered_pred
                )
                
                # Get confusion matrix stats
                for class_id in range(confusion_matrix.matrix.shape[0]):
                    total_tp += confusion_matrix.matrix[class_id][class_id]
                    total_fp += sum(confusion_matrix.matrix[class_id, :]) - confusion_matrix.matrix[class_id][class_id]
                    total_fn += sum(confusion_matrix.matrix[:, class_id]) - confusion_matrix.matrix[class_id][class_id]
            
            # Calculate metrics
            precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
            recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            
            precision_row[conf_threshold] = precision
            recall_row[conf_threshold] = recall
            f1_row[conf_threshold] = f1
        
        precision_results[iou_threshold] = precision_row
        recall_results[iou_threshold] = recall_row
        f1_results[iou_threshold] = f1_row
    
    # Convert results to DataFrames
    precision_df = pd.DataFrame(precision_results).T
    recall_df = pd.DataFrame(recall_results).T
    f1_df = pd.DataFrame(f1_results).T
    
    # Round to 3 decimal places
    precision_df = precision_df.round(3)
    recall_df = recall_df.round(3)
    f1_df = f1_df.round(3)
    
    # Print tables
    print("\nPrecision Table (rows: IoU, columns: Confidence)")
    print(precision_df)
    
    print("\nRecall Table (rows: IoU, columns: Confidence)")
    print(recall_df)
    
    print("\nF1-Score Table (rows: IoU, columns: Confidence)")
    print(f1_df)
    
    # Save to CSV
    precision_df.to_csv('precision_table.csv')
    recall_df.to_csv('recall_table.csv')
    f1_df.to_csv('f1_table.csv')
    
    # Create heatmap visualizations
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    sns.heatmap(precision_df, annot=True, cmap="Blues", fmt=".3f")
    plt.title("Precision")
    
    plt.subplot(1, 3, 2)
    sns.heatmap(recall_df, annot=True, cmap="Blues", fmt=".3f")
    plt.title("Recall")
    
    plt.subplot(1, 3, 3)
    sns.heatmap(f1_df, annot=True, cmap="Blues", fmt=".3f")
    plt.title("F1-Score")
    
    plt.tight_layout()
    plt.savefig('metrics_heatmaps.png')
    plt.show()
    
    return precision_df, recall_df, f1_df

# Create a confusion matrix visualization
def create_confusion_matrix_visualization(test_loader, iou_threshold=0.5, conf_threshold=0.5):
    # Create confusion matrix calculator
    confusion_matrix = ConfusionMatrix(
        num_classes=1,  # Assuming single class detection for solar panels
        iou_threshold=iou_threshold
    )
    
    total_tp = 0
    total_fp = 0
    total_fn = 0
    
    print(f"Creating confusion matrix (IoU={iou_threshold}, conf={conf_threshold})...")
    for img_path in tqdm(test_loader.dataset.img_files):
        # Load the image
        img = load_image(img_path)
        
        # Get ground truth annotations
        label_path = img_path.replace('images', 'labels').replace('.tif', '.txt').replace('.TIF', '.txt')
        gt_boxes = []
        gt_classes = []
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    data = line.strip().split()
                    class_id = int(data[0])
                    x_center, y_center, width, height = map(float, data[1:5])
                    
                    # Convert YOLO format to pixel coordinates
                    img_height, img_width = img.shape[:2]
                    x1 = int((x_center - width/2) * img_width)
                    y1 = int((y_center - height/2) * img_height)
                    x2 = int((x_center + width/2) * img_width)
                    y2 = int((y_center + height/2) * img_height)
                    
                    gt_boxes.append([x1, y1, x2, y2])
                    gt_classes.append(class_id)
        
        # Get model predictions
        results = model(img)
        pred_boxes = []
        pred_scores = []
        pred_classes = []
        
        if results[0].boxes:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            scores = results[0].boxes.conf.cpu().numpy()
            cls = results[0].boxes.cls.cpu().numpy().astype(int)
            
            for box, score, cl in zip(boxes, scores, cls):
                if score >= conf_threshold:
                    pred_boxes.append(box)
                    pred_scores.append(score)
                    pred_classes.append(cl)
        
        # Convert to supervision detections format
        gt_detection = sv.Detections(
            xyxy=np.array(gt_boxes) if gt_boxes else np.zeros((0, 4)),
            class_id=np.array(gt_classes) if gt_classes else np.zeros(0),
            confidence=np.ones(len(gt_classes)) if gt_classes else np.zeros(0)
        )
        
        pred_detection = sv.Detections(
            xyxy=np.array(pred_boxes) if pred_boxes else np.zeros((0, 4)),
            class_id=np.array(pred_classes) if pred_classes else np.zeros(0),
            confidence=np.array(pred_scores) if pred_scores else np.zeros(0)
        )
        
        # Update confusion matrix
        confusion_matrix.process_image(
            gt_detections=gt_detection,
            pred_detections=pred_detection
        )
    
    # Compute overall metrics
    for class_id in range(confusion_matrix.matrix.shape[0]):
        total_tp += confusion_matrix.matrix[class_id][class_id]
        total_fp += sum(confusion_matrix.matrix[class_id, :]) - confusion_matrix.matrix[class_id][class_id]
        total_fn += sum(confusion_matrix.matrix[:, class_id]) - confusion_matrix.matrix[class_id][class_id]
    
    # Calculate metrics
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    # Visualize confusion matrix
    import seaborn as sns
    
    plt.figure(figsize=(10, 8))
    confusion_values = np.array([[total_tp, total_fp], [total_fn, 0]])  # TP, FP, FN, TN (TN is not applicable)
    sns.heatmap(confusion_values, annot=True, fmt="d", cmap="Blues", 
                xticklabels=["Detected", "Not Detected"], 
                yticklabels=["Present", "Not Present"])
    plt.title(f"Confusion Matrix (IoU={iou_threshold}, conf={conf_threshold})\nPrecision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}")
    plt.savefig('confusion_matrix.png')
    plt.show()
    
    print(f"Overall Metrics (IoU={iou_threshold}, conf={conf_threshold}):")
    print(f"True Positives: {total_tp}")
    print(f"False Positives: {total_fp}")
    print(f"False Negatives: {total_fn}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1-Score: {f1:.3f}")
    
    return total_tp, total_fp, total_fn, precision, recall, f1

# Execute the evaluation
if __name__ == "__main__":
    results = evaluate_model()
    print("Evaluation completed!")

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\HP\\Downloads\\labels-20250212T103318Z-001\\labels\\solar_panel_project\\dataset\\labels\\test'

In [23]:
# Modified evaluate_model function
def evaluate_model():
    import seaborn as sns
    
    # Define your test data path - you need to specify this
    test_data_path = r"C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\labels\test"
    
    # Load test data directly instead of through model.datasets
    from ultralytics.data.dataset import YOLODataset
    from ultralytics.data.utils import check_det_dataset
    
    # Get dataset info
    data_dict = check_det_dataset(test_data_path)
    test_dir = data_dict['test']
    
    # Create test dataset
    test_dataset = YOLODataset(
        img_path=test_dir,
        imgsz=640,  # Use the same image size as during training
        batch_size=1,
        augment=False,  # No augmentation for testing
        rect=True,  # Use rectangular inference
        stride=32,
        pad=0.5,
        single_cls=False  # Set to True if you have only one class
    )
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=False
    )
    
    # Rest of your function...
    # 1. Show validation loss convergence
    val_loss = plot_validation_loss()
    
    # 2. Visualize ground truth vs predictions
    visualize_predictions(test_loader)
    
    # 3. Compute mAP50
    map_metrics = compute_map50(test_loader)
    
    # 4. Create P/R/F1 table
    precision_df, recall_df, f1_df = create_metrics_table(test_loader)
    
    # 5. Create a confusion matrix visualization
    create_confusion_matrix_visualization(test_loader, iou_threshold=0.5, conf_threshold=0.5)
    
    return {
        'val_loss': val_loss,
        'map_metrics': map_metrics,
        'precision': precision_df,
        'recall': recall_df,
        'f1': f1_df
    }